<a href="https://colab.research.google.com/github/Mansurbek25/AI_Project/blob/master/Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -c https://repo.continuum.io/miniconda/Miniconda3-py37_4.8.3-Linux-x86_64.sh
!chmod +x Miniconda3-py37_4.8.3-Linux-x86_64.sh
!time bash ./Miniconda3-py37_4.8.3-Linux-x86_64.sh -b -f -p /usr/local
!time conda install -q -y -c conda-forge rdkit

--2020-10-08 07:37:50--  https://repo.continuum.io/miniconda/Miniconda3-py37_4.8.3-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.200.79, 104.18.201.79, 2606:4700::6812:c94f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.200.79|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.3-Linux-x86_64.sh [following]
--2020-10-08 07:37:50--  https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.3-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 88867207 (85M) [application/x-sh]
Saving to: ‘Miniconda3-py37_4.8.3-Linux-x86_64.sh’

Miniconda3-py37_4.8 100%[===================>]  84.75M   222MB/s    in 0.4s    

2020-10-08 07:37:51 (222 MB/s) - ‘Minicon

In [2]:
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

In [17]:
import torch
from torch import nn
import torch.nn.functional as F
import random
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

############# Define a dataloader ################

class Regression_Dataset(Dataset):

    def __init__(self,x_list,y_list):
        self.x_list = x_list
        self.y_list = y_list

    def __len__(self):
        return len(self.x_list)

    def __getitem__(self,idx):
        sample = dict()
        ##############################
        sample['x']=self.x_list[idx]
        sample['y']=self.y_list[idx]
        ##############################
        return sample

############# Define a model #################

class Regression_Model(nn.Module): 
    def __init__(self,num_layer,n_dim):
        ##################################
        super().__init__()
        layers=[nn.Linear(1,n_dim)]
        for i in range(num_layer-1):
          layers.append(nn.Linear(n_dim, n_dim))
        self.linear_layers=nn.ModuleList(layers)
        ###################################
       
            
        

    def forward(self,x):
       ##################################
        linears=self.linear_layers
        for l in linears:
          x=l(x)
        return x
      ##################################
           
        

############# Prepare training set and dataloader for training #############
x_train = np.arange(10000.0)*0.0001 # 10000 numbers from 0 to 1
y_train = x_train ** 2 
train_data = Regression_Dataset(x_train,y_train)
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size = 64, shuffle=True)

###### Initialization: Model, lr, loss, optimizer, learning rate, num_epoch #####

model = Regression_Model(3,1024)
lr = 1e-4
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
num_epoch = 10

model = model.cuda() # Upload to cuda

###### Training part ######

for i in range(num_epoch):
    # Forward propagation and obtain loss
    loss_list = []
    for i_batch, batch in enumerate(train_data_loader):
        x_batch = batch['x'].float()
        y_batch = batch['y'].float()
        x_batch = x_batch.unsqueeze(1)
        y_batch = y_batch.unsqueeze(1) 
        x_batch = x_batch.cuda()
        y_batch = y_batch.cuda()
        y_pred = model(x_batch)
        loss = loss_fn(y_pred,y_batch)
        # Backpropagation and obtain loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step() 
        loss_list.append(loss.data.cpu().numpy())
    avg_loss = np.mean(np.array(loss_list))
    print ('avg loss',avg_loss)


# Test data loader
x_test = np.random.uniform(0,1,5000) # Choose 5000 random numbers between 0 and 1
y_test = x_test ** 2
test_data = Regression_Dataset(x_test,y_test)
test_data_loader  = torch.utils.data.DataLoader(test_data, batch_size=128, shuffle=True)

x_axis = [] # For plotting y = f(x)
y_axis = [] # For plotting y = f(x)


# Validation part

test_loss_list = []
with torch.no_grad():
   for i_batch, batch in enumerate(test_data_loader):
        x_batch = batch['x'].float()
        y_batch = batch['y'].float()
        x_batch = x_batch.unsqueeze(1)
        y_batch = y_batch.unsqueeze(1) 
        x_batch = x_batch.cuda()
        y_batch = y_batch.cuda()
        y_pred = model(x_batch)
        loss = loss_fn(y_pred,y_batch)
        # Backpropagation and obtain loss
        # optimizer.zero_grad()
        # loss.backward()
        # optimizer.step() 
        test_loss_list.append(loss.data.cpu().numpy())
total_loss = np.mean(np.array(test_loss_list))
print ('validity',total_loss)
################################
           # !!!! Implement Here !!!!
################################

# avg_loss = np.mean(np.array(loss_list))

# print ('validity',avg_loss)




/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:445: UserWarning: Using a target size (torch.Size([64, 1])) that is different to the input size (torch.Size([64, 1024])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:445: UserWarning: Using a target size (torch.Size([16, 1])) that is different to the input size (torch.Size([16, 1024])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


avg loss 0.017048117
avg loss 0.005845141
avg loss 0.0057517253
avg loss 0.0057234317
avg loss 0.0058830837
avg loss 0.005777152
avg loss 0.0057632984
avg loss 0.005748319
avg loss 0.0057275966
avg loss 0.005854297
validity 0.0056571723


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:445: UserWarning: Using a target size (torch.Size([128, 1])) that is different to the input size (torch.Size([128, 1024])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:445: UserWarning: Using a target size (torch.Size([8, 1])) that is different to the input size (torch.Size([8, 1024])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
